In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# 제품 직접 링크

In [3]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/slimnuts/products/10208111263'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div/a[2]').click()
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="content"]/div/div[3]/div[4]/ul/li[2]').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
page_total = review_num // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(4)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()}\n" 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)


        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, page]
    time.sleep(1)
df

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div/div[3]/div[4]/ul/li[2]"}
  (Session info: chrome=137.0.7151.55); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff75d826f65+78965]
	GetHandleVerifier [0x0x7ff75d826fc0+79056]
	(No symbol) [0x0x7ff75d5b9dda]
	(No symbol) [0x0x7ff75d6102ea]
	(No symbol) [0x0x7ff75d61059c]
	(No symbol) [0x0x7ff75d663f37]
	(No symbol) [0x0x7ff75d6386af]
	(No symbol) [0x0x7ff75d660dbe]
	(No symbol) [0x0x7ff75d638443]
	(No symbol) [0x0x7ff75d601311]
	(No symbol) [0x0x7ff75d6020a3]
	GetHandleVerifier [0x0x7ff75dade26d+2926461]
	GetHandleVerifier [0x0x7ff75dad8993+2903715]
	GetHandleVerifier [0x0x7ff75daf6aed+3026941]
	GetHandleVerifier [0x0x7ff75d8416fe+187406]
	GetHandleVerifier [0x0x7ff75d8496ef+220159]
	GetHandleVerifier [0x0x7ff75d82faf4+114692]


In [30]:
df.to_csv('형제돼지국밥_1_49.csv', index=False)

In [14]:
df.to_csv('형제돼지국밥_1_9.csv', index=False,)

# 직접 링크 막혔을 때(형제 돼지국밥)

In [5]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/localbybro'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-wholeProductWidget"]/div/div/div/div[2]/ul/li[1]/div/div[1]/a/div[2]').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1) 


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.55)
Stacktrace:
	GetHandleVerifier [0x0x7ff6531d6f65+78965]
	GetHandleVerifier [0x0x7ff6531d6fc0+79056]
	(No symbol) [0x0x7ff652f69dda]
	(No symbol) [0x0x7ff652f55bc5]
	(No symbol) [0x0x7ff652f7ac04]
	(No symbol) [0x0x7ff652ff0195]
	(No symbol) [0x0x7ff6530106cd]
	(No symbol) [0x0x7ff652fe8443]
	(No symbol) [0x0x7ff652fb1311]
	(No symbol) [0x0x7ff652fb20a3]
	GetHandleVerifier [0x0x7ff65348e26d+2926461]
	GetHandleVerifier [0x0x7ff653488993+2903715]
	GetHandleVerifier [0x0x7ff6534a6aed+3026941]
	GetHandleVerifier [0x0x7ff6531f16fe+187406]
	GetHandleVerifier [0x0x7ff6531f96ef+220159]
	GetHandleVerifier [0x0x7ff6531dfaf4+114692]
	GetHandleVerifier [0x0x7ff6531dfca9+115129]
	GetHandleVerifier [0x0x7ff6531c64d8+10728]
	BaseThreadInitThunk [0x0x7ffe1569e8d7+23]
	RtlUserThreadStart [0x0x7ffe166dc5dc+44]


In [ ]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,can2****,25.04.17.,돼지국밥 선택: 맑은국밥(580g 맵지않은) / 포장 패키지 선택: 일반 포장 패키지,"구매자: 거주인원 3인,영유아 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,매번 주문합니다.\n아이가 너무 좋아합니다.\n항상 맛있게 잘 먹고있습니다.,1,0,1
1,anmi****,25.05.15.,돼지국밥 선택: 맑은국밥(580g 맵지않은) / 포장 패키지 선택: 일반 포장 패키지,"구매자: 거주인원 2인,신혼/부부",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,대학생때부터 먹었는데 \n10년이 지났지만 그대로입니다 최곱니다!,1,0,1
2,fghe******,25.04.25.,돼지국밥 선택: 불꽃국밥(620g 신라면맵기) / 포장 패키지 선택: 일반 포장 패키지,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,죄송합니다 이제 가서 안먹어요...밀키트로 먹으니깐 갈 필요가 없어졌습니다 ㅠㅠ 집...,1,0,1
3,toma*****,25.04.14.,돼지국밥 선택: 맑은국밥(580g 맵지않은) / 포장 패키지 선택: 일반 포장 패키지,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 꼼꼼해요,3,가격대비 먹을 만 합니다 배송은 빠릅니다,0,0,1
4,can2****,25.04.17.,돼지국밥 선택: 뽀얀국밥(580g 맵지않은) / 포장 패키지 선택: 일반 포장 패키지,구매자: 거주인원 3인,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,너무 맛있습니다.\n아이가 너무 좋아합니다.,1,0,1
...,...,...,...,...,...,...,...,...,...,...
899,dr34****,21.06.13.,국밥 선택: 돼지국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있습니다 !!!!,1,0,45
900,dong*******,21.06.11.,국밥 선택: 시그니처 불꽃국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛잇어요 좋으네요!!!!!,1,0,46
901,7son*****,21.05.15.,국밥 선택: 돼지국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,고기 완전 많구요 맛납니다!!!!!!!,0,1,46
902,wo****,21.05.12.,국밥 선택: 돼지국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있게 잘먹었습니다,0,3,46


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 904 entries, 0 to 903
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      904 non-null    object
 1   작성일자     904 non-null    object
 2   세부옵션     841 non-null    object
 3   작성자정보    904 non-null    object
 4   간략평가     812 non-null    object
 5   별점       904 non-null    object
 6   리뷰       904 non-null    object
 7   재구매여부    904 non-null    int64 
 8   리뷰 좋아요수  904 non-null    object
 9   페이지      904 non-null    int64 
dtypes: int64(2), object(8)
memory usage: 77.7+ KB


In [ ]:
df.to_csv('형제돼지국밥_리뷰_250520.csv', index=False)

In [ ]:
df.to_excel('형제돼지국밥_리뷰_250520.xlsx', index=False)

# 직접 링크 막혔을 때(건봉 국밥)

In [4]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/gunbong1987'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-bestProductWidget"]/div/div/div/div[2]/ul/li[1]').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1) 


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="pc-bestProductWidget"]/div/div/div/div[2]/ul/li[1]"}
  (Session info: chrome=137.0.7151.55); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff6531d6f65+78965]
	GetHandleVerifier [0x0x7ff6531d6fc0+79056]
	(No symbol) [0x0x7ff652f69dda]
	(No symbol) [0x0x7ff652fc02ea]
	(No symbol) [0x0x7ff652fc059c]
	(No symbol) [0x0x7ff653013f37]
	(No symbol) [0x0x7ff652fe86af]
	(No symbol) [0x0x7ff653010dbe]
	(No symbol) [0x0x7ff652fe8443]
	(No symbol) [0x0x7ff652fb1311]
	(No symbol) [0x0x7ff652fb20a3]
	GetHandleVerifier [0x0x7ff65348e26d+2926461]
	GetHandleVerifier [0x0x7ff653488993+2903715]
	GetHandleVerifier [0x0x7ff6534a6aed+3026941]
	GetHandleVerifier [0x0x7ff6531f16fe+187406]
	GetHandleVerifier [0x0x7ff6531f96ef+220159]
	GetHandleVerifier [0x0x7ff6531

In [16]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,va****,25.05.25.,100%로 국내산 재료로 만든 전국 3대 국밥: 옛순대랑국밥,"구매자: 거주인원 3인,영유아 가정",유통기한 : 아주 넉넉해요 포장 : 별로예요 맛 만족도 : 맛있어요,5,신랑이 순천사람이라 오래전부터 이야기했는데\n냉동식품 못믿는 제가 한참 동안 고민했...,0,0,1
1,smil*****,25.05.09.,100%로 국내산 재료로 만든 전국 3대 국밥: 찰순대랑국밥,정보없음,유통기한 : 꽤 남았어요 포장 : 적당해요 맛 만족도 : 맛있어요,5,유통기한은 안봐서 그냥 적당해요로 했어요\n3월에 순천 놀러가서 윗장에서 먹었던 국...,0,2,1
2,san_*****,25.05.22.,100%로 국내산 재료로 만든 전국 3대 국밥: 건봉 돼지국밥,"구매자: 거주인원 2인,신혼/부부",맛 만족도 : 평범해요 유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요,4,개인적으로 1인이 간단히 먹으면 2번 먹을 양이고\n술안주는 한끼로 딱 맞네요\n살...,0,0,1
3,fall*******,25.05.14.,100%로 국내산 재료로 만든 전국 3대 국밥: 찰순대랑국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,내장국밥 먹어본 후 재 취향이라 순대국밥 2가지와 함께 재구매했어요. 순대국집보다 ...,1,1,1
4,guri****,25.05.10.,100%로 국내산 재료로 만든 전국 3대 국밥: 건봉 돼지국밥,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,순천건봉국밥 처음 먹어보는데 맛있고 양도 많네요. 종류별로 주문후 하나씩 맛보는 중...,0,0,1
...,...,...,...,...,...,...,...,...,...,...
19995,kjc5***,22.12.28.,내맘대로 선택하는 9가지 국밥: 찰순대랑국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있게 잘 먹고있어요. 다 먹고 다시 주문할게요,0,1,1000
19996,ku****,23.01.04.,내맘대로 선택하는 9가지 국밥: 새끼보국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,재구매 냄새없고 진짜맛있어요,0,1,1000
19997,cosm*****,22.12.14.,내맘대로 선택하는 9가지 국밥: 건봉국밥,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 꼼꼼해요,3,맛은 그런대로 나쁘지 않습니다,0,2,1000
19998,a112****,22.12.28.,내맘대로 선택하는 9가지 국밥: 건봉국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,재구매했습니다ㅎㅎㅎ\n건봉덕에 사먹는 동네국밥은 안쳐다보게 되네요ㅎㅎ,1,1,1000


In [17]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,va****,25.05.25.,100%로 국내산 재료로 만든 전국 3대 국밥: 옛순대랑국밥,"구매자: 거주인원 3인,영유아 가정",유통기한 : 아주 넉넉해요 포장 : 별로예요 맛 만족도 : 맛있어요,5,신랑이 순천사람이라 오래전부터 이야기했는데\n냉동식품 못믿는 제가 한참 동안 고민했...,0,0,1
1,smil*****,25.05.09.,100%로 국내산 재료로 만든 전국 3대 국밥: 찰순대랑국밥,정보없음,유통기한 : 꽤 남았어요 포장 : 적당해요 맛 만족도 : 맛있어요,5,유통기한은 안봐서 그냥 적당해요로 했어요\n3월에 순천 놀러가서 윗장에서 먹었던 국...,0,2,1
2,san_*****,25.05.22.,100%로 국내산 재료로 만든 전국 3대 국밥: 건봉 돼지국밥,"구매자: 거주인원 2인,신혼/부부",맛 만족도 : 평범해요 유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요,4,개인적으로 1인이 간단히 먹으면 2번 먹을 양이고\n술안주는 한끼로 딱 맞네요\n살...,0,0,1
3,fall*******,25.05.14.,100%로 국내산 재료로 만든 전국 3대 국밥: 찰순대랑국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,내장국밥 먹어본 후 재 취향이라 순대국밥 2가지와 함께 재구매했어요. 순대국집보다 ...,1,1,1
4,guri****,25.05.10.,100%로 국내산 재료로 만든 전국 3대 국밥: 건봉 돼지국밥,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,순천건봉국밥 처음 먹어보는데 맛있고 양도 많네요. 종류별로 주문후 하나씩 맛보는 중...,0,0,1
...,...,...,...,...,...,...,...,...,...,...
19995,kjc5***,22.12.28.,내맘대로 선택하는 9가지 국밥: 찰순대랑국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있게 잘 먹고있어요. 다 먹고 다시 주문할게요,0,1,1000
19996,ku****,23.01.04.,내맘대로 선택하는 9가지 국밥: 새끼보국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,재구매 냄새없고 진짜맛있어요,0,1,1000
19997,cosm*****,22.12.14.,내맘대로 선택하는 9가지 국밥: 건봉국밥,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 꼼꼼해요,3,맛은 그런대로 나쁘지 않습니다,0,2,1000
19998,a112****,22.12.28.,내맘대로 선택하는 9가지 국밥: 건봉국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,재구매했습니다ㅎㅎㅎ\n건봉덕에 사먹는 동네국밥은 안쳐다보게 되네요ㅎㅎ,1,1,1000


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      20000 non-null  object
 1   작성일자     20000 non-null  object
 2   세부옵션     20000 non-null  object
 3   작성자정보    20000 non-null  object
 4   간략평가     19951 non-null  object
 5   별점       20000 non-null  object
 6   리뷰       20000 non-null  object
 7   재구매여부    20000 non-null  int64 
 8   리뷰 좋아요수  20000 non-null  object
 9   페이지      20000 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 1.7+ MB


In [19]:
df.to_csv('건봉국밥_리뷰_250527.csv', index=False)

# 직접 링크 막혔을 때(대건명가) - brand 사이트

In [51]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://brand.naver.com/dgmg'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-customProductWidget"]/div/div/div/div/div[1]/div[1]/div/div/div/div[1]/a').click()
time.sleep(1)
driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div/a[2]').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for pages_10 in range(1, page_total // 10 + 1):
    for cnt in range(1, 11):
        page = (pages_10 - 1) * 10 + cnt
        print(f'page {page} 수집 시작')
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{cnt+1}]').click()
        time.sleep(1) 


        soup = BeautifulSoup(driver.page_source, 'html.parser')

        for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
            writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
            score = content.find('div', class_='_3i1mVq_JBd').find('em').text
            review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
            date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
            if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
                repurchase = 0
            else:
                repurchase = 1

            try:    
                consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
                consumer_info += ': '
                for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                    consumer_info += i.text + ','
                consumer_info = consumer_info[:-1]
                
            except:
                consumer_info = '정보없음'


            eval_info = ''
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
                subject = i.find('dt')
                answer = i.find('dd')
                eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
                
            product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
            for word in ['구매자', '유통기한', '맛 만족도']:
                product_info = product_info.split(word)[0]
            
            # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
            review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

            df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]
    time.sleep(1)
    driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[12]').click()
    time.sleep(2)



page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [52]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,mm****,25.05.14.,,"구매자: 거주인원 5인,대가족",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,1. 가격이 저렴하다.\n처음 이 돼지국밥을 접하게 된 계기는 우연히 온라인 쇼핑몰...,0,1,1
1,noin****,25.05.21.,,구매자: 거주인원 4인,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"대건명가의 돼지국밥 밀키트를 주문해봤는데, 이건 정말 부산 현지 맛집을 집으로 그대...",0,0,1
2,www9***,25.05.09.,,"구매자: 거주인원 1인,싱글,식이관심사 다이어트",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,🍲 해장엔 역시 이 맛! 대건명가 엄마가 보낸 국밥 밀키트 후기 (★5점)\n술 마...,1,0,1
3,vulb****,25.05.23.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,국물이 전문점처럼 아주 진해요.\n누린내가 아주 없다고 할 순 없지만 돼지국밥 특성...,0,0,1
4,jony****,25.05.22.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,너무 많이 구매해서 리뷰꺼리가 소진되었는데 그래도 써볼랍니다 ^^\n\n대건명가 돼...,1,0,1
...,...,...,...,...,...,...,...,...,...,...
16595,qsom***,23.07.13.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,4,세일할때 잘 샀어요,0,0,830
16596,poca****,23.07.11.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,냄새도 없고 국물도 시원하고 맛있어요,0,0,830
16597,tjsg******,23.07.14.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,말해 모해요~ 몇년째 주문해서 먹고 있어요~ 아이들 이제 방학이라~ 먹을것없을...,0,0,830
16598,wpg6******,23.07.12.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,항상 맛있고 잡내안나요,0,0,830


In [54]:
df[df.duplicated()]

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
145,elle****,25.05.16.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,떨어져가고 있어요!!\n없으면 불안한 대건명가예요!!\n라방만 기다리고 있습니다!...,1,0,8
418,ajoo******,25.05.22.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,배송빠르고 상품좋아요,1,0,21
554,elle****,25.04.29.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,주말단골메뉴입니다! 한그릇씩 후딱주고 가게로 가는데요 . 저한테는 없어서는 안될...,1,0,28
605,toya***,25.05.23.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,믿고먹는대건돼지국밥ㅋ,1,0,31
652,kv****,25.05.22.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,대폭 할인된...\n포장도 완벽...,0,0,33
...,...,...,...,...,...,...,...,...,...,...
15798,haha****,23.08.17.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있어서 믿고 먹고 있어요,1,0,790
15824,leeh*******,23.08.16.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 꽤 남았어요 포장 : 적당해요 맛 만족도 : 맛있어요,5,선물했는데 맛있다고 합니다 감사합니다,1,0,792
16023,nana*****,23.08.18.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,행사때마다 항상 쟁여두고 있어요~ 가족들은 물론 시부모님도 맛있다고 칭찬하시네요^^,1,0,802
16279,rufl****,23.07.16.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,냄새안나고 양많아서 좋아요 고기도 부드럽구요,0,0,814


In [55]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,mm****,25.05.14.,NaN,"구매자: 거주인원 5인,대가족",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,1. 가격이 저렴하다.\n처음 이 돼지국밥을 접하게 된 계기는 우연히 온라인 쇼핑몰...,0,1,1
1,noin****,25.05.21.,NaN,구매자: 거주인원 4인,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"대건명가의 돼지국밥 밀키트를 주문해봤는데, 이건 정말 부산 현지 맛집을 집으로 그대...",0,0,1
2,www9***,25.05.09.,NaN,"구매자: 거주인원 1인,싱글,식이관심사 다이어트",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,🍲 해장엔 역시 이 맛! 대건명가 엄마가 보낸 국밥 밀키트 후기 (★5점)\n술 마...,1,0,1
3,vulb****,25.05.23.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,국물이 전문점처럼 아주 진해요.\n누린내가 아주 없다고 할 순 없지만 돼지국밥 특성...,0,0,1
4,jony****,25.05.22.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,너무 많이 구매해서 리뷰꺼리가 소진되었는데 그래도 써볼랍니다 ^^\n\n대건명가 돼...,1,0,1
...,...,...,...,...,...,...,...,...,...,...
16595,qsom***,23.07.13.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,4,세일할때 잘 샀어요,0,0,830
16596,poca****,23.07.11.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,냄새도 없고 국물도 시원하고 맛있어요,0,0,830
16597,tjsg******,23.07.14.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,말해 모해요~ 몇년째 주문해서 먹고 있어요~ 아이들 이제 방학이라~ 먹을것없을...,0,0,830
16598,wpg6******,23.07.12.,메뉴선택: 1.돼지국밥4팩+얼큰1인분증정,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,항상 맛있고 잡내안나요,0,0,830


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16600 entries, 0 to 16599
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      16600 non-null  object
 1   작성일자     16600 non-null  object
 2   세부옵션     11506 non-null  object
 3   작성자정보    16600 non-null  object
 4   간략평가     16600 non-null  object
 5   별점       16600 non-null  object
 6   리뷰       16600 non-null  object
 7   재구매여부    16600 non-null  int64 
 8   리뷰 좋아요수  16600 non-null  object
 9   페이지      16600 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 1.4+ MB


In [57]:
df.to_csv('대건명가_리뷰_250527.csv', index=False)

# 직접 링크 막혔을 때(도야진국)

In [58]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/doyajin9'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-wholeProductWidget"]/div/div/div/ul/li[1]/div/a/div[2]/div/div/div/img').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1) 


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [59]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,kimm***,25.04.30.,돼지국밥3종(돼지/순대/섞어): 돼지국밥5팩,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"택배는 받자마자 바로 뜯어보았고, \n포장은 아이스팩 두개 넣어져서 잘왔어요\n근데...",1,1,1
1,yahi****,25.05.26.,돼지국밥3종(돼지/순대/섞어): 돼지국밥3팩+순대국밥1팩+섞어국밥1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,캠핑용 음식 아침대용으로 딱이네요\n고기도 많고 양도 많고 넉넉한 파까지 센스 짱이네요,0,0,1
2,raco****,25.05.26.,돼지국밥3종(돼지/순대/섞어): 돼지국밥5팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,매번 주문해서 먹네요\n아이들 밥에 저의 점심으로 먹다보니 빨리 소진ㄷ히네요 ㅎㅎ\...,1,0,1
3,sexy*****,25.05.27.,돼지국밥3종(돼지/순대/섞어): 돼지국밥5팩,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,돼지국밥집보다 더 맛있어요. 고기양은 얼마나 많은지.. 계속 재주문하는 국밥맛집이에요,1,0,1
4,jje0***,25.05.23.,돼지국밥3종(돼지/순대/섞어): 돼지국밥3팩+순대국밥2팩,"구매자: 거주인원 4인,영유아 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있어서 여러번 재주문 했어요.\n특히 깍두기가 정말 맛있어요^^ 국내산 고춧가루 ...,1,0,1
...,...,...,...,...,...,...,...,...,...,...
14583,sexy******,19.01.11.,국밥 묶음: 돼지국밥5팩+다대기1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,포장도 배송도 맛도 완벽합니다 자주이용할거같아요,0,0,730
14584,jung*****,19.01.09.,국밥 묶음: 돼지국밥5팩+다대기1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있고 배달까지해줘 너무 좋아요,0,0,730
14585,lavi****,19.01.02.,국밥 묶음: 돼지국밥5팩+다대기1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,깔끔하고 진한맛이예요~,0,0,730
14586,yoon****,19.01.24.,국밥 묶음: 돼지국밥5팩+다대기1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,너무너무 맛있어요^^,0,0,730


In [60]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,kimm***,25.04.30.,돼지국밥3종(돼지/순대/섞어): 돼지국밥5팩,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"택배는 받자마자 바로 뜯어보았고, \n포장은 아이스팩 두개 넣어져서 잘왔어요\n근데...",1,1,1
1,yahi****,25.05.26.,돼지국밥3종(돼지/순대/섞어): 돼지국밥3팩+순대국밥1팩+섞어국밥1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,캠핑용 음식 아침대용으로 딱이네요\n고기도 많고 양도 많고 넉넉한 파까지 센스 짱이네요,0,0,1
2,raco****,25.05.26.,돼지국밥3종(돼지/순대/섞어): 돼지국밥5팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,매번 주문해서 먹네요\n아이들 밥에 저의 점심으로 먹다보니 빨리 소진ㄷ히네요 ㅎㅎ\...,1,0,1
3,sexy*****,25.05.27.,돼지국밥3종(돼지/순대/섞어): 돼지국밥5팩,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,돼지국밥집보다 더 맛있어요. 고기양은 얼마나 많은지.. 계속 재주문하는 국밥맛집이에요,1,0,1
4,jje0***,25.05.23.,돼지국밥3종(돼지/순대/섞어): 돼지국밥3팩+순대국밥2팩,"구매자: 거주인원 4인,영유아 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있어서 여러번 재주문 했어요.\n특히 깍두기가 정말 맛있어요^^ 국내산 고춧가루 ...,1,0,1
...,...,...,...,...,...,...,...,...,...,...
14583,sexy******,19.01.11.,국밥 묶음: 돼지국밥5팩+다대기1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,포장도 배송도 맛도 완벽합니다 자주이용할거같아요,0,0,730
14584,jung*****,19.01.09.,국밥 묶음: 돼지국밥5팩+다대기1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있고 배달까지해줘 너무 좋아요,0,0,730
14585,lavi****,19.01.02.,국밥 묶음: 돼지국밥5팩+다대기1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,깔끔하고 진한맛이예요~,0,0,730
14586,yoon****,19.01.24.,국밥 묶음: 돼지국밥5팩+다대기1팩,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,너무너무 맛있어요^^,0,0,730


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14588 entries, 0 to 14587
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      14588 non-null  object
 1   작성일자     14588 non-null  object
 2   세부옵션     12451 non-null  object
 3   작성자정보    14588 non-null  object
 4   간략평가     14588 non-null  object
 5   별점       14588 non-null  object
 6   리뷰       14588 non-null  object
 7   재구매여부    14588 non-null  int64 
 8   리뷰 좋아요수  14588 non-null  object
 9   페이지      14588 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 1.2+ MB


In [62]:
df.to_csv('도야진국_리뷰_250527.csv', index=False)

# 직접 링크 막혔을 때(명장김치찌개)

In [41]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/slimnuts'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-newProductWidget"]/div/div/div/div/div/ul/li[1]/div/div[1]/a/div').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1) 


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [51]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,song*******,25.05.29.,수량: 츄니토리X명장김치찌개 10개입,"구매자: 거주인원 4인,취학자녀 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"이건 진짜 맛있고 해먹기도 편해요, 근데 잘라져 있어서 찜으로 먹기엔 뭔가 찢어먹는...",1,0,1
1,kimj*****,25.05.11.,수량: 츄니토리X명장김치찌개 5개입,"구매자: 거주인원 1인,싱글",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,너무 맛있어요 ! 엄청 간단한데 꿀맛이라 더 좋네요 ! 츄니믿고 구매했는데 역시나 ...,0,0,1
2,myin*****,25.05.16.,수량: 명장김치찌개 4개입,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,재료좋고 맛도 있고요\n아이디어상품인거 같아요\n이렇게 만들어서(저는 못만듬ㅋㅋ) ...,1,0,1
3,pk****,25.05.30.,수량: 츄니토리X명장김치찌개 10개입,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 다이어트",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,푹 지진 김치찜 입니다 \n집에서 만들기 어려운 맛이에요 \n첨 먹고 맛있어서 냉동...,0,0,1
4,jscj*****,25.05.23.,수량: 츄니토리X명장김치찌개 5개입,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,성분을 들여다보면 살짝 맘에안드는부분이 몇가지 있긴하지만 ㅎㅎ 그러려니 하고 드실거...,1,0,1
...,...,...,...,...,...,...,...,...,...,...
9048,mell*****,24.05.02.,수량: 4개입포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,찌개나 찜 하려면 김치맛이 중요한데\n김치가 넘 맛있어서 두부만 넣어도\n요리가되고...,0,0,453
9049,kohy*******,24.04.28.,수량: 4개입,구매자: 거주인원 2인,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,사골국에 이거 하나만 풀어서 넣고\n두부만 썰어넣으면 완벽~~! 너무 맛있고\n간편...,0,0,453
9050,gy****,24.04.22.,수량: 4개입,정보없음,맛 만족도 : 평범해요 포장 : 적당해요,5,숙성김치라그런지 진짜 일반 김치찌개끓일때랑 다르네요\n감칠맛...장난아닙니다👍👍\n...,0,0,453
9051,wjsa******,24.04.26.,수량: 4개입포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,배달어플로 김치찜 자주시키는데 집에서 간편하게 조리할수 있으니 좋네요 물만 좀 넣어...,0,0,453


In [52]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,song*******,25.05.29.,수량: 츄니토리X명장김치찌개 10개입,"구매자: 거주인원 4인,취학자녀 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"이건 진짜 맛있고 해먹기도 편해요, 근데 잘라져 있어서 찜으로 먹기엔 뭔가 찢어먹는...",1,0,1
1,kimj*****,25.05.11.,수량: 츄니토리X명장김치찌개 5개입,"구매자: 거주인원 1인,싱글",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,너무 맛있어요 ! 엄청 간단한데 꿀맛이라 더 좋네요 ! 츄니믿고 구매했는데 역시나 ...,0,0,1
2,myin*****,25.05.16.,수량: 명장김치찌개 4개입,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,재료좋고 맛도 있고요\n아이디어상품인거 같아요\n이렇게 만들어서(저는 못만듬ㅋㅋ) ...,1,0,1
3,pk****,25.05.30.,수량: 츄니토리X명장김치찌개 10개입,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 다이어트",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,푹 지진 김치찜 입니다 \n집에서 만들기 어려운 맛이에요 \n첨 먹고 맛있어서 냉동...,0,0,1
4,jscj*****,25.05.23.,수량: 츄니토리X명장김치찌개 5개입,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,성분을 들여다보면 살짝 맘에안드는부분이 몇가지 있긴하지만 ㅎㅎ 그러려니 하고 드실거...,1,0,1
...,...,...,...,...,...,...,...,...,...,...
9048,mell*****,24.05.02.,수량: 4개입포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,찌개나 찜 하려면 김치맛이 중요한데\n김치가 넘 맛있어서 두부만 넣어도\n요리가되고...,0,0,453
9049,kohy*******,24.04.28.,수량: 4개입,구매자: 거주인원 2인,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,사골국에 이거 하나만 풀어서 넣고\n두부만 썰어넣으면 완벽~~! 너무 맛있고\n간편...,0,0,453
9050,gy****,24.04.22.,수량: 4개입,정보없음,맛 만족도 : 평범해요 포장 : 적당해요,5,숙성김치라그런지 진짜 일반 김치찌개끓일때랑 다르네요\n감칠맛...장난아닙니다👍👍\n...,0,0,453
9051,wjsa******,24.04.26.,수량: 4개입포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,배달어플로 김치찜 자주시키는데 집에서 간편하게 조리할수 있으니 좋네요 물만 좀 넣어...,0,0,453


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9053 entries, 0 to 9052
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      9053 non-null   object
 1   작성일자     9053 non-null   object
 2   세부옵션     9053 non-null   object
 3   작성자정보    9053 non-null   object
 4   간략평가     9053 non-null   object
 5   별점       9053 non-null   object
 6   리뷰       9053 non-null   object
 7   재구매여부    9053 non-null   int64 
 8   리뷰 좋아요수  9053 non-null   object
 9   페이지      9053 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 778.0+ KB


In [54]:
df.to_csv('명장김치찌개_리뷰_250602.csv', index=False)

# 직접 링크 막혔을 때(신사강 김치찌개)

In [36]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함
# 헤더 추가

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/davincistyle'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-bestProductWidget"]/div/div/div/div[2]/div[1]/div/ul/li[2]/div/div[1]/a/div').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1) 


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [37]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,goun****,25.05.26.,제품선택: 김치찌개_900g(3-4인분),정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,김치찌개 김치찜 다 맛있어요\n텁텁하지 않고 개운한 맛이예요\n목살 넣거나 꽁치 넣...,1,0,1
1,youj*****,25.05.27.,제품선택: 김치찜_500g(1-2인분),정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,냉동실에 항상 쟁여놓고 먹는 \n김치찜 너무 맛있어요.\n잘 먹겠습니다.,1,0,1
2,youj*****,25.05.27.,제품선택: 김치찌개_500g(2인분),정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,냉장고 놓고 먹어도 여전히 맛있어요.\n잘 먹겠습니다.,1,0,1
3,sua2******,25.06.02.,제품선택: (김치볶음)만능김볶_200g(2인분),정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 적당해요,3,물건잘받았습니다 먹어보겠습니다,1,0,1
4,sua2******,25.06.02.,제품선택: 김치찌개_500g(2인분),정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 적당해요,3,물건잘받았습니다 먹어보겠습니다,1,0,1
...,...,...,...,...,...,...,...,...,...,...
6354,skki***,14.08.04.,제품선택: 01.매운맛(오리지날)_500g,정보없음,,5,잘받았어요,0,0,318
6355,imag*****,14.08.05.,제품선택: 01.매운맛(오리지날)_500g,정보없음,,5,건승하세요,0,0,318
6356,dkfm*******,14.08.07.,제품선택: 02.약간매운맛_500g,정보없음,,5,ㅅ극시니디ㅢ읻ㅅㄴㅅㅇㄴㄷ,0,0,318
6357,ansg******,14.03.20.,,정보없음,,5,깔끔하이 좋아요~ 맛보구 맛나면 또시킬게요 ㅎ,0,0,318


In [38]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,goun****,25.05.26.,제품선택: 김치찌개_900g(3-4인분),정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,김치찌개 김치찜 다 맛있어요\n텁텁하지 않고 개운한 맛이예요\n목살 넣거나 꽁치 넣...,1,0,1
1,youj*****,25.05.27.,제품선택: 김치찜_500g(1-2인분),정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,냉동실에 항상 쟁여놓고 먹는 \n김치찜 너무 맛있어요.\n잘 먹겠습니다.,1,0,1
2,youj*****,25.05.27.,제품선택: 김치찌개_500g(2인분),정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,냉장고 놓고 먹어도 여전히 맛있어요.\n잘 먹겠습니다.,1,0,1
3,sua2******,25.06.02.,제품선택: (김치볶음)만능김볶_200g(2인분),정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 적당해요,3,물건잘받았습니다 먹어보겠습니다,1,0,1
4,sua2******,25.06.02.,제품선택: 김치찌개_500g(2인분),정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 적당해요,3,물건잘받았습니다 먹어보겠습니다,1,0,1
...,...,...,...,...,...,...,...,...,...,...
6354,skki***,14.08.04.,제품선택: 01.매운맛(오리지날)_500g,정보없음,NaN,5,잘받았어요,0,0,318
6355,imag*****,14.08.05.,제품선택: 01.매운맛(오리지날)_500g,정보없음,NaN,5,건승하세요,0,0,318
6356,dkfm*******,14.08.07.,제품선택: 02.약간매운맛_500g,정보없음,NaN,5,ㅅ극시니디ㅢ읻ㅅㄴㅅㅇㄴㄷ,0,0,318
6357,ansg******,14.03.20.,NaN,정보없음,NaN,5,깔끔하이 좋아요~ 맛보구 맛나면 또시킬게요 ㅎ,0,0,318


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6359 entries, 0 to 6358
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      6359 non-null   object
 1   작성일자     6359 non-null   object
 2   세부옵션     6325 non-null   object
 3   작성자정보    6359 non-null   object
 4   간략평가     4164 non-null   object
 5   별점       6359 non-null   object
 6   리뷰       6359 non-null   object
 7   재구매여부    6359 non-null   int64 
 8   리뷰 좋아요수  6359 non-null   object
 9   페이지      6359 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 546.5+ KB


In [40]:
df.to_csv('신사강김치찌개_리뷰_250602.csv', index=False)

# 직접 링크 막혔을 때(금돼지식당) - shopping 

In [ ]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://shopping.naver.com/window-products/emart/5898830485'
driver.get(url)

time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#_productTabContainer > div > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1)


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [ ]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

C:\Users\user\AppData\Local\Temp\ipykernel_45404\1477857550.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,khs3***,25.04.29.,NaN,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,냉장보관이라 원래 유통기한이 많이 길지는 않네요.\n\n건더기는 김치반 고기반 두부...,0,0,1
1,zeri***,25.05.28.,NaN,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,세일해서 여러개 쟁입니다. 김치도 고기도 맛있고 건더기가 큼직해서 좋아요!! 두부 ...,1,0,1
2,naye******,25.06.01.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,한 끼 간편하게 먹기 좋아요 내용물도 많이 들어있고 맛있어요,0,0,1
3,quan****,25.05.27.,NaN,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 적당해요,4,"돼지고기 김치찌개를 워낙 좋아해서 주문해봤어요. 아직 먹어보진 않았는데요, 시식후에...",0,0,1
4,koi4***,25.06.02.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,꾸준히 구입하는 김치찌개에요 무난하게 맛있습니다,1,0,1
...,...,...,...,...,...,...,...,...,...,...
4096,mira********,21.10.19.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있게 잘먹었습니다 ^^,0,0,205
4097,ebon***,21.10.27.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,4,요거 맛있네요.재구매 의사있습니다,0,0,205
4098,jmjr***,21.10.31.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있네요 한끼 든든하게 먹었어요,0,0,205
4099,282c******,21.10.27.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,후기좋아서 샀는데 김치찌개 제품중에 제일 나은거 같아요,0,0,205


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4101 entries, 0 to 4100
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   작성자      4101 non-null   object 
 1   작성일자     4101 non-null   object 
 2   세부옵션     0 non-null      float64
 3   작성자정보    4101 non-null   object 
 4   간략평가     4101 non-null   object 
 5   별점       4101 non-null   object 
 6   리뷰       4101 non-null   object 
 7   재구매여부    4101 non-null   int64  
 8   리뷰 좋아요수  4101 non-null   object 
 9   페이지      4101 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 352.4+ KB


In [ ]:
df.to_csv('금돼지식당_250602.csv', index=False)

# 직접 링크 막혔을 때(수백당)

In [5]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함
# 헤더 추가

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/subackdang'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-bestProductWidget"]/div/div/div/div[2]/div[1]/div/ul/li[1]').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1) 


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [6]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,19bo*****,25.06.04.,선택: [수백당] 얼큰순대국,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,리뉴얼 된후 처음 구매하고 어떤맛일까 진짜 궁금했는데~ 이거 완전 물건이네용^^혹시...,1,3,1
1,jskj******,25.05.29.,선택: [수백당] 돼지국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,수백당 국밥이 맛있다고 해서 구매했어요~~^^\n일단 포장꼼꼼하게 파손없이 잘왔구...,0,3,1
2,qkrr*******,25.05.19.,선택: [수백당] 돼지국밥,"구매자: 거주인원 5인,대가족",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,저희 아버지가 평소에 국밥을 정말 좋아하세요 ^^\n특히 수백당 매장 국밥을 참 좋...,0,0,1
3,ysyo******,25.05.31.,선택: [수백당] 돼지국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,저번에 얼큰순대국밥을 주문해보고 맛있어서 국밥중의 기본인 돼지국밥을 주문했어요 수백...,0,2,1
4,blue******,25.05.19.,선택: [수백당] 돼지국밥,"구매자: 거주인원 4인,신혼/부부,식이관심사 저탄고지",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,와이프가 장모님이랑 여행 가면서 주문해줬어요.\n2박 3일동안 혼자 집에서 먹었습니...,0,0,1
...,...,...,...,...,...,...,...,...,...,...
12607,cjsw*****,20.12.15.,품목: 돼지국밥 570ml,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,추천! 추천! 추천!,1,0,631
12608,dldy********,20.11.16.,품목: 돼지국밥 570ml,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,국밥 참 시원하네요,0,1,631
12609,ro****,20.12.09.,,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 적당해요,4,잘받았습니다 잘먹겠습니다,0,1,631
12610,09****,20.12.02.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,좋습니다~~~~~~,0,1,631


In [7]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,19bo*****,25.06.04.,선택: [수백당] 얼큰순대국,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,리뉴얼 된후 처음 구매하고 어떤맛일까 진짜 궁금했는데~ 이거 완전 물건이네용^^혹시...,1,3,1
1,jskj******,25.05.29.,선택: [수백당] 돼지국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,수백당 국밥이 맛있다고 해서 구매했어요~~^^\n일단 포장꼼꼼하게 파손없이 잘왔구...,0,3,1
2,qkrr*******,25.05.19.,선택: [수백당] 돼지국밥,"구매자: 거주인원 5인,대가족",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,저희 아버지가 평소에 국밥을 정말 좋아하세요 ^^\n특히 수백당 매장 국밥을 참 좋...,0,0,1
3,ysyo******,25.05.31.,선택: [수백당] 돼지국밥,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,저번에 얼큰순대국밥을 주문해보고 맛있어서 국밥중의 기본인 돼지국밥을 주문했어요 수백...,0,2,1
4,blue******,25.05.19.,선택: [수백당] 돼지국밥,"구매자: 거주인원 4인,신혼/부부,식이관심사 저탄고지",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,와이프가 장모님이랑 여행 가면서 주문해줬어요.\n2박 3일동안 혼자 집에서 먹었습니...,0,0,1
...,...,...,...,...,...,...,...,...,...,...
12607,cjsw*****,20.12.15.,품목: 돼지국밥 570ml,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,추천! 추천! 추천!,1,0,631
12608,dldy********,20.11.16.,품목: 돼지국밥 570ml,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,국밥 참 시원하네요,0,1,631
12609,ro****,20.12.09.,NaN,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 적당해요,4,잘받았습니다 잘먹겠습니다,0,1,631
12610,09****,20.12.02.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,좋습니다~~~~~~,0,1,631


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12612 entries, 0 to 12611
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      12612 non-null  object
 1   작성일자     12612 non-null  object
 2   세부옵션     5260 non-null   object
 3   작성자정보    12612 non-null  object
 4   간략평가     12568 non-null  object
 5   별점       12612 non-null  object
 6   리뷰       12612 non-null  object
 7   재구매여부    12612 non-null  int64 
 8   리뷰 좋아요수  12612 non-null  object
 9   페이지      12612 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 1.1+ MB


In [10]:
df.to_csv('./data/수백당_리뷰_250606.csv', index=False)

# 직접 링크 막혔을 때(꼰대국밥)

In [46]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/ilsa'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-newProductWidget"]/div/div/div/div/div/ul/li[2]/div/div[1]/a/div').click()
time.sleep(3)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

driver.find_element(By.CSS_SELECTOR, '#content > div > div._3CTsMZymJs > div:nth-child(3) > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1) 


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [47]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,pheg***,25.05.31.,수량: 12개,"구매자: 거주인원 2인,신혼/부부,식이관심사 다이어트,저탄고지,저속노화",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,역시 현존 냉동 돼지국밥 중 최고입니다 ㅎㅎㅎ 다진마늘 조금 + 다대기 + 부추 넣...,1,0,1
1,0228***,25.05.19.,수량: 12개,정보없음,유통기한 : 꽤 남았어요 포장 : 적당해요 맛 만족도 : 맛있어요,5,배송오는 과정중에 스티로폼 상단이 파손되어 윗부분 국밥이 살짝 녹아있어서 아쉬웠지만...,1,0,1
2,goun****,25.05.27.,수량: 8개,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,식당에서 파는 것보다 더 깔끔하고 맛있어요\n늘 만족합니다,0,0,1
3,happ*******,25.05.21.,,"구매자: 거주인원 4인,대가족,식이관심사 다이어트",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,다른 국밥은 맛이 없어서 다시 주문했어요. 조리 편하고 맛있어요.,1,0,1
4,dhap***,25.05.19.,수량: 12개,구매자: 거주인원 3인,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,돼지국밥중단연최고인것같습니다! 맛있어요\n재구매계속하고있네요ㅎㅎ,1,0,1
...,...,...,...,...,...,...,...,...,...,...
8126,wint******,17.06.09.,,정보없음,,5,믿고 사는 가지산 돼지국밥!,0,0,407
8127,psyc*******,17.06.16.,,정보없음,,5,맛있습니다.,0,0,407
8128,khj8******,17.05.14.,,정보없음,,5,잘받았습니다 감사합니다,0,0,407
8129,car8***,16.08.17.,,정보없음,,5,서비스도 넣어주신 사장님 깜짝 서비스로 전복을 넣어주셔서 넣어서 끓였습니다\n감사합...,0,0,407


In [48]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,pheg***,25.05.31.,수량: 12개,"구매자: 거주인원 2인,신혼/부부,식이관심사 다이어트,저탄고지,저속노화",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,역시 현존 냉동 돼지국밥 중 최고입니다 ㅎㅎㅎ 다진마늘 조금 + 다대기 + 부추 넣...,1,0,1
1,0228***,25.05.19.,수량: 12개,정보없음,유통기한 : 꽤 남았어요 포장 : 적당해요 맛 만족도 : 맛있어요,5,배송오는 과정중에 스티로폼 상단이 파손되어 윗부분 국밥이 살짝 녹아있어서 아쉬웠지만...,1,0,1
2,goun****,25.05.27.,수량: 8개,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,식당에서 파는 것보다 더 깔끔하고 맛있어요\n늘 만족합니다,0,0,1
3,happ*******,25.05.21.,NaN,"구매자: 거주인원 4인,대가족,식이관심사 다이어트",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,다른 국밥은 맛이 없어서 다시 주문했어요. 조리 편하고 맛있어요.,1,0,1
4,dhap***,25.05.19.,수량: 12개,구매자: 거주인원 3인,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,돼지국밥중단연최고인것같습니다! 맛있어요\n재구매계속하고있네요ㅎㅎ,1,0,1
...,...,...,...,...,...,...,...,...,...,...
8126,wint******,17.06.09.,NaN,정보없음,NaN,5,믿고 사는 가지산 돼지국밥!,0,0,407
8127,psyc*******,17.06.16.,NaN,정보없음,NaN,5,맛있습니다.,0,0,407
8128,khj8******,17.05.14.,NaN,정보없음,NaN,5,잘받았습니다 감사합니다,0,0,407
8129,car8***,16.08.17.,NaN,정보없음,NaN,5,서비스도 넣어주신 사장님 깜짝 서비스로 전복을 넣어주셔서 넣어서 끓였습니다\n감사합...,0,0,407


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8131 entries, 0 to 8130
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      8131 non-null   object
 1   작성일자     8131 non-null   object
 2   세부옵션     36 non-null     object
 3   작성자정보    8131 non-null   object
 4   간략평가     5819 non-null   object
 5   별점       8131 non-null   object
 6   리뷰       8131 non-null   object
 7   재구매여부    8131 non-null   int64 
 8   리뷰 좋아요수  8131 non-null   object
 9   페이지      8131 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 698.8+ KB


In [50]:
df.to_csv('./data/꼰대국밥_리뷰_250607.csv', index=False)

# 직접 링크 막혔을 때(장사의신)

In [41]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함
# 헤더 추가

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/jangsinmall'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-customProductWidget1"]/div/div/div/div[2]/div/div/ul/li[2]/div/div[1]/a').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1) 


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [42]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,gt****,25.05.03.,선택하기: 돼지국밥 230g,"구매자: 거주인원 2인,싱글,식이관심사 저속노화",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,대한민국에서 가장 저렴한 거 같아요. 만들자마자 바로 오니 신선도는 말할 것도 없고...,1,10,1
1,nogf******,25.05.24.,선택하기: 얼큰 순대국밥 255g,"구매자: 거주인원 4인,취학자녀 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,여러가지 시킨거 3일만에 받음.\n유튜브의 장신만 봐오면서 매번 라이브때마다 이벤트...,0,1,1
2,gots******,25.05.14.,선택하기: 순대국밥 240g,"구매자: 거주인원 2인,식이관심사 저당",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"국을 조아하는 아들땜,\n한 여름에도 준비해야 해요😅😅\n여름엔 미역냉국,갈비탕,백...",1,1,1
3,gots******,25.05.30.,선택하기: 얼큰 돼지국밥 245g,"구매자: 거주인원 2인,식이관심사 저당",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"장신몰 밀키트를 최소 5개 이상,\n쟁여둬야 하는 강박증이 생겼어요^^;;ㅎ\n\n...",1,1,1
4,gots******,25.05.14.,선택하기: 얼큰 돼지국밥 245g,"구매자: 거주인원 2인,식이관심사 저당",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"국을 조아하는 아들땜,\n한 여름에도 준비해야 해요😅😅\n여름엔 미역냉국,갈비탕,백...",1,1,1
...,...,...,...,...,...,...,...,...,...,...
6777,kkok********,25.02.05.,선택하기: 돼지국밥 210g,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,아주조아요아주조아요,0,1,339
6778,debo*******,25.02.10.,선택하기: 순대국밥 210g,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 꼼꼼해요,4,많이파세요 많이파세요,0,0,339
6779,llss*****,25.02.08.,선택하기: 얼큰 돼지국밥 225g,"구매자: 거주인원 2인,신혼/부부",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있습니다......,0,1,339
6780,cher****,25.02.09.,선택하기: 얼큰 순대국밥 225g,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,asdgasdfsdfsdfsdsd,0,1,340


In [43]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,gt****,25.05.03.,선택하기: 돼지국밥 230g,"구매자: 거주인원 2인,싱글,식이관심사 저속노화",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,대한민국에서 가장 저렴한 거 같아요. 만들자마자 바로 오니 신선도는 말할 것도 없고...,1,10,1
1,nogf******,25.05.24.,선택하기: 얼큰 순대국밥 255g,"구매자: 거주인원 4인,취학자녀 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,여러가지 시킨거 3일만에 받음.\n유튜브의 장신만 봐오면서 매번 라이브때마다 이벤트...,0,1,1
2,gots******,25.05.14.,선택하기: 순대국밥 240g,"구매자: 거주인원 2인,식이관심사 저당",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"국을 조아하는 아들땜,\n한 여름에도 준비해야 해요😅😅\n여름엔 미역냉국,갈비탕,백...",1,1,1
3,gots******,25.05.30.,선택하기: 얼큰 돼지국밥 245g,"구매자: 거주인원 2인,식이관심사 저당",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"장신몰 밀키트를 최소 5개 이상,\n쟁여둬야 하는 강박증이 생겼어요^^;;ㅎ\n\n...",1,1,1
4,gots******,25.05.14.,선택하기: 얼큰 돼지국밥 245g,"구매자: 거주인원 2인,식이관심사 저당",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"국을 조아하는 아들땜,\n한 여름에도 준비해야 해요😅😅\n여름엔 미역냉국,갈비탕,백...",1,1,1
...,...,...,...,...,...,...,...,...,...,...
6777,kkok********,25.02.05.,선택하기: 돼지국밥 210g,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,아주조아요아주조아요,0,1,339
6778,debo*******,25.02.10.,선택하기: 순대국밥 210g,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 꼼꼼해요,4,많이파세요 많이파세요,0,0,339
6779,llss*****,25.02.08.,선택하기: 얼큰 돼지국밥 225g,"구매자: 거주인원 2인,신혼/부부",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있습니다......,0,1,339
6780,cher****,25.02.09.,선택하기: 얼큰 순대국밥 225g,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,asdgasdfsdfsdfsdsd,0,1,340


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6782 entries, 0 to 6781
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      6782 non-null   object
 1   작성일자     6782 non-null   object
 2   세부옵션     6782 non-null   object
 3   작성자정보    6782 non-null   object
 4   간략평가     6782 non-null   object
 5   별점       6782 non-null   object
 6   리뷰       6782 non-null   object
 7   재구매여부    6782 non-null   int64 
 8   리뷰 좋아요수  6782 non-null   object
 9   페이지      6782 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 582.8+ KB


In [45]:
df.to_csv('./data/장사의신_리뷰_250607.csv', index=False)

# 직접 링크 막혔을 때(몽실종가) - brand 사이트

In [34]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://brand.naver.com/local'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-bestProductWidget"]/div/div/div/div[2]/div/div[1]/ul/li[4]/div/div[2]/a').click()
time.sleep(1)
driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div/a[2]').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for pages_10 in range(1, page_total // 10 + 1):
    for cnt in range(1, 11):
        page = (pages_10 - 1) * 10 + cnt
        print(f'page {page} 수집 시작')
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{cnt+1}]').click()
        time.sleep(1) 


        soup = BeautifulSoup(driver.page_source, 'html.parser')

        for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
            writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
            score = content.find('div', class_='_3i1mVq_JBd').find('em').text
            review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
            date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
            if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
                repurchase = 0
            else:
                repurchase = 1

            try:    
                consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
                consumer_info += ': '
                for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                    consumer_info += i.text + ','
                consumer_info = consumer_info[:-1]
                
            except:
                consumer_info = '정보없음'


            eval_info = ''
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
                subject = i.find('dt')
                answer = i.find('dd')
                eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
                
            product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
            for word in ['구매자', '유통기한', '맛 만족도']:
                product_info = product_info.split(word)[0]
            
            # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
            review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

            df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]
    time.sleep(1)
    driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[12]').click()
    time.sleep(2)



page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [35]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,k990****,25.05.31.,메뉴 선택: 돼지국밥 1인분,"구매자: 거주인원 1인,싱글",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,우선 포장이 너무 깔끔한 개별포장이라서 마음에 듭니다. 잡내에 민감한데 잡내 없이 ...,0,0,1
1,o1o1***,25.05.24.,메뉴 선택: 돼지국밥 1인분,"구매자: 거주인원 2인,신혼/부부",유통기한 : 아주 넉넉해요 포장 : 적당해요 맛 만족도 : 맛있어요,5,집앞 국밥집보다 훨씬 맛있었어요^^! 살짝 냄새가 나나 싶은 느낌은 있었지만 또 돈...,0,0,1
2,ghdf***,25.05.13.,메뉴 선택: 돼지국밥 1인분,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,4,"세일할때 마다 주저없이 구매,비축하는 밀키트입니다\n고기를 추가로 구매해서 더넣어서...",0,0,1
3,youy********,25.06.06.,메뉴 선택: 돼지국밥 1인분,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,고향이 부산인데 생각나서 시켜봤어요~~\n자주먹던 그맛이라 너무 만족스러워요♡♡\n...,0,0,1
4,jina****,25.06.04.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,몇번이고 재구매합니다! 돼지국밥 중 몽실종가가 제일 맛있어요~ 냉동실 쟁여템 강추에요!!,0,0,1
...,...,...,...,...,...,...,...,...,...,...
4395,didr******,23.05.16.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,4,맛있어요~ 잘먹을께요!,0,1,220
4396,kmj8****,23.05.18.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,저렴한 가격에 잘 구매했어용,0,1,220
4397,jmy9***,23.05.21.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있어오ㅡ깔끔하게 포장되어,0,1,220
4398,nu****,23.05.06.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 적당해요 맛 만족도 : 맛있어요,5,처음 시킨곳인데 아이들이 맛있다고 몇번을 말하네요. 계속 시켜먹을것 같아요~^^,0,1,220


In [36]:
df[df.duplicated()]

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
25,chlr****,25.06.06.,메뉴 선택: 돼지국밥 1인분,"구매자: 거주인원 4인,취학자녀 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있어요 아이들이 좋아해서 자주 이용하고 있어요,1,0,2
130,eofl********,25.04.22.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,존맛탱구리 몽실종가 돼지국밥... \n매운 다데기가 있었으면 좋겠다는 생각이 들지만...,1,0,7
131,eofl********,25.04.22.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,존맛탱구리 몽실종가 돼지국밥... \n매운 다데기가 있었으면 좋겠다는 생각이 들지만...,1,0,7
392,dalk*******,25.01.11.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,너무맛있어요 ~늘 여기국밥만 먹어요!!!,1,0,20
436,meti***,25.01.04.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,배송빠르고 맘에들어요,1,0,22
1262,jkgg****,24.06.26.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,유튜브 보고 구매했는데 실제로 맛있고 잡내도 안 나서 좋아요! 포장도 깔끔하게 왔었...,1,0,64
1545,kjjh****,24.06.12.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 꽤 남았어요 포장 : 적당해요 맛 만족도 : 맛있어요,3,배송이 너무 오래 걸렸네요.,1,0,78
2003,wbdl****,24.02.26.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있어서 자주 구매했어요\n근데 갑자기 파가 없어져서 아쉬워요\n질도 조금씩 떨어지...,1,0,101
2331,cxd2***,24.01.05.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,배송 잘 왔어요 항상 먹는거라 딱히 쓸내용은 없는데 그냥 항상 구매해서 먹고 있습니다.,1,0,117
3073,bawo****,23.10.04.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,진짜진짜진짜 맛있어요,1,0,154


In [37]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,k990****,25.05.31.,메뉴 선택: 돼지국밥 1인분,"구매자: 거주인원 1인,싱글",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,우선 포장이 너무 깔끔한 개별포장이라서 마음에 듭니다. 잡내에 민감한데 잡내 없이 ...,0,0,1
1,o1o1***,25.05.24.,메뉴 선택: 돼지국밥 1인분,"구매자: 거주인원 2인,신혼/부부",유통기한 : 아주 넉넉해요 포장 : 적당해요 맛 만족도 : 맛있어요,5,집앞 국밥집보다 훨씬 맛있었어요^^! 살짝 냄새가 나나 싶은 느낌은 있었지만 또 돈...,0,0,1
2,ghdf***,25.05.13.,메뉴 선택: 돼지국밥 1인분,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,4,"세일할때 마다 주저없이 구매,비축하는 밀키트입니다\n고기를 추가로 구매해서 더넣어서...",0,0,1
3,youy********,25.06.06.,메뉴 선택: 돼지국밥 1인분,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,고향이 부산인데 생각나서 시켜봤어요~~\n자주먹던 그맛이라 너무 만족스러워요♡♡\n...,0,0,1
4,jina****,25.06.04.,메뉴 선택: 돼지국밥 1인분,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,몇번이고 재구매합니다! 돼지국밥 중 몽실종가가 제일 맛있어요~ 냉동실 쟁여템 강추에요!!,0,0,1
...,...,...,...,...,...,...,...,...,...,...
4395,didr******,23.05.16.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,4,맛있어요~ 잘먹을께요!,0,1,220
4396,kmj8****,23.05.18.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,저렴한 가격에 잘 구매했어용,0,1,220
4397,jmy9***,23.05.21.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있어오ㅡ깔끔하게 포장되어,0,1,220
4398,nu****,23.05.06.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 적당해요 맛 만족도 : 맛있어요,5,처음 시킨곳인데 아이들이 맛있다고 몇번을 말하네요. 계속 시켜먹을것 같아요~^^,0,1,220


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4400 entries, 0 to 4399
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      4400 non-null   object
 1   작성일자     4400 non-null   object
 2   세부옵션     2620 non-null   object
 3   작성자정보    4400 non-null   object
 4   간략평가     4400 non-null   object
 5   별점       4400 non-null   object
 6   리뷰       4400 non-null   object
 7   재구매여부    4400 non-null   int64 
 8   리뷰 좋아요수  4400 non-null   object
 9   페이지      4400 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 378.1+ KB


In [40]:
df.to_csv('./data/몽실종가_리뷰_250607.csv', index=False)

# 직접 링크 막혔을 때(프레시지 불고기) - brand 사이트

In [3]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://brand.naver.com/fresheasy/products/2482069991'
driver.get(url)

# driver.find_element(By.XPATH, '//*[@id="pc-bestProductWidget"]/div/div/div/div[2]/div/div[1]/ul/li[4]/div/div[2]/a').click()
# time.sleep(1)
# driver.find_element(By.XPATH, '//*[@id="content"]/div/div/div/a[2]').click()
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for pages_10 in range(1, page_total // 10 + 1):
    for cnt in range(1, 11):
        page = (pages_10 - 1) * 10 + cnt
        print(f'page {page} 수집 시작')
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{cnt+1}]').click()
        time.sleep(1) 


        soup = BeautifulSoup(driver.page_source, 'html.parser')

        for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
            writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
            score = content.find('div', class_='_3i1mVq_JBd').find('em').text
            review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
            date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
            if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
                repurchase = 0
            else:
                repurchase = 1

            try:    
                consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
                consumer_info += ': '
                for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                    consumer_info += i.text + ','
                consumer_info = consumer_info[:-1]
                
            except:
                consumer_info = '정보없음'


            eval_info = ''
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
                subject = i.find('dt')
                answer = i.find('dd')
                eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
                
            product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
            for word in ['구매자', '유통기한', '맛 만족도']:
                product_info = product_info.split(word)[0]
            
            # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
            review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

            df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]
    time.sleep(1)
    driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[12]').click()
    time.sleep(2)



page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [4]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,zz****,25.05.26.,,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,간편하게 해먹을 수 있고 맛있어요,0,0,1
1,dltn******,25.05.19.,,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,2인분이라 하기엔 양이 조금 적은 듯하구요.. 맛은 있어요..,0,0,1
2,cu****,25.05.16.,,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 꼼꼼해요,5,간편하게 조리해서 맛있게 먹었어요,0,0,1
3,ss_s****,25.06.05.,,"구매자: 거주인원 2인,신혼/부부,식이관심사 다이어트,저당,고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,여행가서 맛있게 잘 먹었습니다,1,0,1
4,na****,25.05.30.,,"구매자: 거주인원 3인,취학자녀 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"프레시지 밀키트는 늘 재구매해서 먹었답니다. 편리하고, 한끼로 3인 식사 가능합니다.",0,0,1
...,...,...,...,...,...,...,...,...,...,...
5595,flsk*****,18.03.09.,수령일 선택: 3월 6일(화),정보없음,,5,맛있어여~,0,0,280
5596,spar********,18.02.25.,수령일 선택: 2월 21일(수),정보없음,,3,배송도 빠르고 포장도 잘되어 왔어요,0,0,280
5597,asul****,18.03.25.,수령일 선택: 3월 24일(토),정보없음,,5,맛있습니다,0,0,280
5598,rhu3***,18.03.03.,수령일 선택: 2월 28일(수),정보없음,,5,잘받았습니다,0,0,280


In [5]:
df[df.duplicated()]

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
528,jumi****,22.07.27.,,정보없음,,5,너모너모 맛있어요 여러 번 시켜 먹고 있어요,1,0,27
975,cwpa******,21.10.14.,포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,진짜 짱좋아요 킹왕짱 최고!!,1,0,49
976,cwpa******,21.10.14.,포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,진짜 짱좋아요 킹왕짱 최고!!,1,0,49
977,cwpa******,21.10.14.,포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,진짜 짱좋아요 킹왕짱 최고!!,1,0,49
1047,peop*****,21.09.23.,포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,굿굿굿짱짱 아주좋아요 ㅎㅎ,0,0,53
1050,jung****,21.08.25.,포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,회사 랜선회식으로 주문했는데 다 만족하시고 맛있었어요!,1,0,53
1062,cwpa******,21.09.12.,포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,품질도 좋구 배송두빨라요,1,0,54
1063,cwpa******,21.09.12.,포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,품질도 좋구 배송두빨라요,1,0,54
1064,cwpa******,21.09.12.,포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,품질도 좋구 배송두빨라요,1,0,54
1068,cwpa******,21.09.12.,포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛도좋구 편리하고 포장두깔끔해요,1,0,54


In [6]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,zz****,25.05.26.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,간편하게 해먹을 수 있고 맛있어요,0,0,1
1,dltn******,25.05.19.,NaN,정보없음,유통기한 : 꽤 남았어요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,2인분이라 하기엔 양이 조금 적은 듯하구요.. 맛은 있어요..,0,0,1
2,cu****,25.05.16.,NaN,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 꼼꼼해요,5,간편하게 조리해서 맛있게 먹었어요,0,0,1
3,ss_s****,25.06.05.,NaN,"구매자: 거주인원 2인,신혼/부부,식이관심사 다이어트,저당,고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,여행가서 맛있게 잘 먹었습니다,1,0,1
4,na****,25.05.30.,NaN,"구매자: 거주인원 3인,취학자녀 가정",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"프레시지 밀키트는 늘 재구매해서 먹었답니다. 편리하고, 한끼로 3인 식사 가능합니다.",0,0,1
...,...,...,...,...,...,...,...,...,...,...
5595,flsk*****,18.03.09.,수령일 선택: 3월 6일(화),정보없음,NaN,5,맛있어여~,0,0,280
5596,spar********,18.02.25.,수령일 선택: 2월 21일(수),정보없음,NaN,3,배송도 빠르고 포장도 잘되어 왔어요,0,0,280
5597,asul****,18.03.25.,수령일 선택: 3월 24일(토),정보없음,NaN,5,맛있습니다,0,0,280
5598,rhu3***,18.03.03.,수령일 선택: 2월 28일(수),정보없음,NaN,5,잘받았습니다,0,0,280


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5600 entries, 0 to 5599
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      5600 non-null   object
 1   작성일자     5600 non-null   object
 2   세부옵션     3339 non-null   object
 3   작성자정보    5600 non-null   object
 4   간략평가     4241 non-null   object
 5   별점       5600 non-null   object
 6   리뷰       5600 non-null   object
 7   재구매여부    5600 non-null   int64 
 8   리뷰 좋아요수  5600 non-null   object
 9   페이지      5600 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 481.2+ KB


In [8]:
df.to_csv('./data/프레시지불고기_리뷰_250607.csv', index=False)

# 직접 링크 막혔을 때(동추원 소불고기)

In [22]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함
# 헤더 추가

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://smartstore.naver.com/ourbrand'
driver.get(url)

driver.find_element(By.XPATH, '//*[@id="pc-customProductWidget2"]/div/div/div/div[2]/div/div/ul/li[4]/div/div[1]/a/div').click()
time.sleep(3)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
driver.find_element(By.CSS_SELECTOR, '#content > div > div._3CTsMZymJs > div:nth-child(4) > div._27jmWaPaKy > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1) 


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작
page 18 수집 시작
page 19 수집 시작
page 20 수집 시작
page 21 수집 시작
page 22 수집 시작
page 23 수집 시작
page 24 수집 시작
page 25 수집 시작
page 26 수집 시작
page 27 수집 시작
page 28 수집 시작
page 29 수집 시작
page 30 수집 시작
page 31 수집 시작
page 32 수집 시작
page 33 수집 시작
page 34 수집 시작
page 35 수집 시작
page 36 수집 시작
page 37 수집 시작
page 38 수집 시작
page 39 수집 시작
page 40 수집 시작
page 41 수집 시작
page 42 수집 시작
page 43 수집 시작
page 44 수집 시작
page 45 수집 시작
page 46 수집 시작
page 47 수집 시작
page 48 수집 시작
page 49 수집 시작
page 50 수집 시작
page 51 수집 시작
page 52 수집 시작
page 53 수집 시작
page 54 수집 시작
page 55 수집 시작
page 56 수집 시작
page 57 수집 시작
page 58 수집 시작
page 59 수집 시작
page 60 수집 시작
page 61 수집 시작
page 62 수집 시작
page 63 수집 시작
page 64 수집 시작
page 65 수집 시작
page 66 수집 시작
page 67 수집 시작
page 68 수집 시작
page 69 수집 시작
page 70 수집 시작
page 71 수집 시작
page 72 수집 시작
p

In [23]:
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,thsd******,25.05.28.,용량: 410g / 수량: 4개,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,벌써 다섯번째 주문일거예요 우리집 가족들이 모두 잘먹는 제품이라 냉동고 쟁여템이에요...,1,0,1
1,luxu*******,25.05.19.,용량: 410g / 수량: 1개,"구매자: 거주인원 4인,싱글",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,들어있는게 간단하게 들어있어서\n맛을 의심했는데ㅎㅎ 버섯야채 추가넣고\n넣으라는 물...,0,0,1
2,rosa*******,25.05.20.,용량: 410g / 수량: 4개,구매자: 거주인원 4인,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"너무 맛있네요~~ 대파, 양파, 팽이버섯 넣고 같이 끓여서 넉넉하게 먹었습니다. 우...",0,0,1
3,kara****,25.06.01.,용량: 410g / 수량: 4개,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,두번째 주문입니다. 고기도 좋고 맛있어요. 채소만 더 사서 곁드리면 언제나 만족스런...,0,0,1
4,syba***,25.06.03.,용량: 410g / 수량: 4개,구매자: 거주인원 5인,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,항상 냉동실에 쟁여놓는 불고기예요.\n버섯 등 야채추가하면 아주 훌륭한 한끼 차림이...,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2080,stef*******,20.03.13.,토요일수령: 가능포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,4,신선하고 맛있게 먹었어요~,0,0,105
2081,hy****,20.03.10.,토요일수령: 가능포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,첨 먹어보는데 정말 맛있네요 ㅎㅎㅎ,0,0,105
2082,eu****,20.03.18.,토요일 수령 가능 여부: 가능포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있어요!!!!!!,1,0,105
2083,ssol***,20.03.10.,토요일수령: 가능포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,편리하게 먹기 좋아요.,0,0,105


In [24]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,thsd******,25.05.28.,용량: 410g / 수량: 4개,"구매자: 거주인원 4인,취학자녀 가정,식이관심사 고단백",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,벌써 다섯번째 주문일거예요 우리집 가족들이 모두 잘먹는 제품이라 냉동고 쟁여템이에요...,1,0,1
1,luxu*******,25.05.19.,용량: 410g / 수량: 1개,"구매자: 거주인원 4인,싱글",유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,들어있는게 간단하게 들어있어서\n맛을 의심했는데ㅎㅎ 버섯야채 추가넣고\n넣으라는 물...,0,0,1
2,rosa*******,25.05.20.,용량: 410g / 수량: 4개,구매자: 거주인원 4인,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,"너무 맛있네요~~ 대파, 양파, 팽이버섯 넣고 같이 끓여서 넉넉하게 먹었습니다. 우...",0,0,1
3,kara****,25.06.01.,용량: 410g / 수량: 4개,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,두번째 주문입니다. 고기도 좋고 맛있어요. 채소만 더 사서 곁드리면 언제나 만족스런...,0,0,1
4,syba***,25.06.03.,용량: 410g / 수량: 4개,구매자: 거주인원 5인,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,항상 냉동실에 쟁여놓는 불고기예요.\n버섯 등 야채추가하면 아주 훌륭한 한끼 차림이...,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2080,stef*******,20.03.13.,토요일수령: 가능포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,4,신선하고 맛있게 먹었어요~,0,0,105
2081,hy****,20.03.10.,토요일수령: 가능포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,첨 먹어보는데 정말 맛있네요 ㅎㅎㅎ,0,0,105
2082,eu****,20.03.18.,토요일 수령 가능 여부: 가능포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있어요!!!!!!,1,0,105
2083,ssol***,20.03.10.,토요일수령: 가능포장꼼꼼해요,정보없음,포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,편리하게 먹기 좋아요.,0,0,105


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2085 entries, 0 to 2084
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   작성자      2085 non-null   object
 1   작성일자     2085 non-null   object
 2   세부옵션     1427 non-null   object
 3   작성자정보    2085 non-null   object
 4   간략평가     1788 non-null   object
 5   별점       2085 non-null   object
 6   리뷰       2085 non-null   object
 7   재구매여부    2085 non-null   int64 
 8   리뷰 좋아요수  2085 non-null   object
 9   페이지      2085 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 179.2+ KB


In [26]:
df.to_csv('./data/동추원소불고기_리뷰_250607.csv', index=False)

# 직접 링크 막혔을 때(홈밀 소불고기) - shopping 

In [27]:
df = pd.DataFrame(columns=['작성자', '작성일자', '세부옵션', '작성자정보', '간략평가',  '별점', '리뷰', '재구매여부', '리뷰 좋아요수', '페이지'])


options = Options()
# options.add_argument('--headless')  # 브라우저 창 없이 실행
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver = webdriver.Chrome(options=options)

# 2. 대상 페이지 접속
url = 'https://shopping.naver.com/window-products/homeplus/6258608392'
driver.get(url)

time.sleep(3)
driver.find_element(By.CSS_SELECTOR, '#_productTabContainer > div > ul > li:nth-child(2) > a').click()
time.sleep(1)

review_num = driver.find_element(By.XPATH, '//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]/a/span').text
if ',' in review_num:
    review_num = review_num.replace(',', '')
page_total = int(review_num) // 20 + 1

for page in range(1, page_total+1):
    print(f'page {page} 수집 시작')
    if page == 1:
        pass

    elif page <= 5:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div/div[3]/div[2]/div/div/a[{page+1}]').click()
        time.sleep(1) 

    elif (page_total - page) <= 3:
        add_idx = 4 - (page_total - page)
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div/div[3]/div[2]/div/div/a[{7+add_idx}]').click()
        time.sleep(1)


    else:
        driver.find_element(By.XPATH, f'//*[@id="REVIEW"]/div/div/div[3]/div[2]/div/div/a[7]').click()
        time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    for content in soup.find_all('li', class_='BnwL_cs1av _nlog_click _nlog_impression_element'):
        writer = content.find('div', class_='_3i1mVq_JBd').find('strong').text
        score = content.find('div', class_='_3i1mVq_JBd').find('em').text
        review = content.find('div', class_='_1kMfD5ErZ6').find('span', class_='_2L3vDiadT9').text
        date = content.find('div', class_='_3i1mVq_JBd').find('span', class_="_2L3vDiadT9").text
        if content.find('div', class_='_1kMfD5ErZ6').find('span', class_='byXA4FP1Bq') is None:
            repurchase = 0
        else:
            repurchase = 1

        try:    
            consumer_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find('dt').text
            consumer_info += ': '
            for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find('div', '_3F8sJXhFeW').find_all('span'):
                consumer_info += i.text + ','
            consumer_info = consumer_info[:-1]
            
        except:
            consumer_info = '정보없음'


        eval_info = ''
        for i in content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").find_all('div', class_ = '_1QLwBCINAr'):
            subject = i.find('dt')
            answer = i.find('dd')
            eval_info += f"{subject.text.strip()} : {answer.text.strip()} " 
            
        product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(strip=True)
        for word in ['구매자', '유통기한', '맛 만족도']:
            product_info = product_info.split(word)[0]
        
        # product_info = content.find('div', class_='_3i1mVq_JBd').find('div', class_="_2FXNMst_ak").get_text(separator=' ', strip=True)
        review_good = content.find('div', class_='_2UOli-XSOY').find('span', class_='count').text

        df.loc[len(df)] = [writer, date, product_info , consumer_info, eval_info, score, review, repurchase, review_good, page]

page 1 수집 시작
page 2 수집 시작
page 3 수집 시작
page 4 수집 시작
page 5 수집 시작
page 6 수집 시작
page 7 수집 시작
page 8 수집 시작
page 9 수집 시작
page 10 수집 시작
page 11 수집 시작
page 12 수집 시작
page 13 수집 시작
page 14 수집 시작
page 15 수집 시작
page 16 수집 시작
page 17 수집 시작


In [28]:
# 빈 텍스트를 결측치로 변환
df = df.replace(r'^\s*$', np.nan, regex=True)
df

C:\Users\user\AppData\Local\Temp\ipykernel_30500\1477857550.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'^\s*$', np.nan, regex=True)


,작성자,작성일자,세부옵션,작성자정보,간략평가,별점,리뷰,재구매여부,리뷰 좋아요수,페이지
0,wjds*******,25.05.14.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,저는 물을 좀 자작하게 더 부어서 끓였어요\n편하고 맛도 있고 좋아요!,0,0,1
1,ddal****,25.05.22.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,추천해요 양은 적고요 반찬없을때 넘 유용해요,0,0,1
2,whwd*******,25.05.27.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,맛있게 잘먹겠습니다,1,0,1
3,gind*****,25.05.21.,NaN,정보없음,유통기한 : 아주 넉넉해요 포장 : 꼼꼼해요 맛 만족도 : 맛있어요,5,정확한 배송\n신뢰가는 제품\n너무 좋아요,0,0,1
4,wpdu*****,25.05.16.,NaN,정보없음,유통기한 : 임박상품이에요 포장 : 적당해요 맛 만족도 : 맛있어요,5,밀키트 간편해서 좋아요,0,0,1
...,...,...,...,...,...,...,...,...,...,...
326,yunj******,22.05.24.,NaN,정보없음,맛 만족도 : 평범해요 유통기한 : 꽤 남았어요 포장 : 적당해요,5,잘 먹엇어요 편리해요,1,0,17
327,iam_***,22.05.10.,NaN,정보없음,NaN,4,잘 받았습니다 ~~~,0,0,17
328,wate*******,22.04.14.,NaN,정보없음,NaN,5,강추합니다. 간편하게 조리해 먹을수있는 최상의 상태로 잘 배달되었습니다. 맛도 좋고...,0,0,17
329,jisu****,22.04.08.,NaN,정보없음,NaN,5,한번 먹고 나니 없네요...,0,0,17


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 331 entries, 0 to 330
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   작성자      331 non-null    object 
 1   작성일자     331 non-null    object 
 2   세부옵션     0 non-null      float64
 3   작성자정보    331 non-null    object 
 4   간략평가     270 non-null    object 
 5   별점       331 non-null    object 
 6   리뷰       331 non-null    object 
 7   재구매여부    331 non-null    int64  
 8   리뷰 좋아요수  331 non-null    object 
 9   페이지      331 non-null    int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 28.4+ KB


In [31]:
df.to_csv('./data/홈밀소불고기_리뷰_250602.csv', index=False)